In [3]:
import os
import pandas as pd


In [8]:
import zipfile
import pandas as pd
import os

folderpath = r"../data_raw/2022"
filepaths = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

dfs = []

for f in filepaths:
    if f.endswith(".zip"):
        with zipfile.ZipFile(f) as z:
            csv_files = [name for name in z.namelist() if name.endswith(".csv")]
            if len(csv_files) == 0:
                print(f"No CSV found in {f}")
                continue
            with z.open(csv_files[0]) as my_file:
                # 🔧 Fix: specify encoding fallback
                dfs.append(pd.read_csv(my_file, encoding="utf-8", on_bad_lines="skip", low_memory=False))
    else:
        # 🔧 Fix here as well
        dfs.append(pd.read_csv(f, encoding="utf-8", on_bad_lines="skip", low_memory=False))

df = pd.concat(dfs, ignore_index=True)

print("Shape:", df.shape)



Shape: (895485, 14)


In [9]:
Token = "HnJYIWmbZvvBYxPTkxeJJHtFgAEPKCQK"


In [10]:
import os
Token = os.getenv("NOAA_TOKEN")


In [13]:
import os
print(os.getcwd())


/Users/boukaskasbrahim/Desktop/Newyork_citibike/notebooks


In [16]:
from dotenv import load_dotenv
import os

# 👇 Explicitly tell dotenv to load from the parent directory
load_dotenv(dotenv_path="../.env")

Token = os.getenv("NOAA_TOKEN")
print("Token loaded:", bool(Token))


Token loaded: True


In [ ]:
import requests
import json

r = requests.get(
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"
    "datasetid=GHCND&datatypeid=TAVG&limit=1000&"
    "stationid=GHCND:USW00014739&startdate=2022-01-01&enddate=2022-12-31",
    headers={"token": Token}
)

d = json.loads(r.text)

print(" Status code:", r.status_code)
print("Number of results:", len(d.get('results', [])))


 Status code: 200
Number of results: 365


In [19]:
from datetime import datetime
import pandas as pd

# Filter only average temperature results
avg_temps = [item for item in d['results'] if item['datatype'] == 'TAVG']

# Extract the dates and temperature values
dates_temp = [item['date'] for item in avg_temps]
temps = [item['value'] for item in avg_temps]

# Create a DataFrame
df_temps = pd.DataFrame()
df_temps['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temps['avgTemp'] = [float(v)/10.0 for v in temps]  # convert tenths of °C to °C

df_temps.head()


,date,avgTemp
0,2022-01-01,7.7
1,2022-01-02,7.4
2,2022-01-03,-1.4
3,2022-01-04,-4.9
4,2022-01-05,2.8


In [20]:
# Ensure started_at is in datetime format
df['started_at'] = pd.to_datetime(df['started_at'])

# Extract the date (without time) for each trip
df['date'] = df['started_at'].dt.date

# Convert df_temps date column to just the date (no time)
df_temps['date'] = pd.to_datetime(df_temps['date']).dt.date

# Merge both DataFrames by date
df_merged = df.merge(df_temps, how='left', on='date', indicator=True)

# Check merge result
df_merged['_merge'].value_counts(dropna=False)


_merge
both          895485
left_only          0
right_only         0
Name: count, dtype: int64

In [21]:
df_merged.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Unnamed: 0,date,avgTemp,_merge
0,919C40A703A965D7,electric_bike,2022-04-15 15:02:20,2022-04-15 15:57:16,Pershing Field,JC024,Pershing Field,JC024,40.742677,-74.051789,40.742677,-74.051789,casual,NaN,2022-04-15,12.6,both
1,3B40831921DAD6C1,classic_bike,2022-04-08 10:20:55,2022-04-08 10:25:29,JC Medical Center,JC011,Grand St,JC102,40.716540,-74.049638,40.715178,-74.037683,member,NaN,2022-04-08,11.4,both
2,69C5C0766309F73D,electric_bike,2022-04-09 14:14:04,2022-04-09 14:18:51,Mama Johnson Field - 4 St & Jackson St,HB404,Southwest Park - Jackson St & Observer Hwy,HB401,40.743140,-74.040041,40.737551,-74.041664,member,NaN,2022-04-09,10.5,both
3,304B82055A1D4A24,classic_bike,2022-04-05 19:43:37,2022-04-05 19:51:14,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735938,-74.030305,40.740802,-74.042521,casual,NaN,2022-04-05,10.1,both
4,1CB760ECBCB08609,classic_bike,2022-04-17 11:25:26,2022-04-17 11:29:45,Mama Johnson Field - 4 St & Jackson St,HB404,Mama Johnson Field - 4 St & Jackson St,HB404,40.743140,-74.040041,40.743140,-74.040041,member,NaN,2022-04-17,7.8,both


In [22]:
df_merged.to_csv("../data_raw/citibike_weather_2022.csv", index=False)
print(" Merged data saved successfully!")


 Merged data saved successfully!
